In [2]:
import os

file_path = os.path.abspath('Testing.ipynb')
file_dir = os.path.dirname(file_path)

print("File path:", file_path)
print("Directory:", file_dir)

File path: C:\Users\timlf\PycharmProjects\Semester_Project_Too\Testing.ipynb
Directory: C:\Users\timlf\PycharmProjects\Semester_Project_Too


In [5]:
import pyautogui
pyautogui.click(2000, 600)
pyautogui.write(f"python /home/ipa/quanz/user_accounts/tfessler/software/pyRetLIFE/scripts/create_spectrum.py --config ~/config_files/VH2O_create_spectrum.yaml")
# pyautogui.press('enter')


In [21]:
import os
print("Exists:", os.path.exists("config_files"))
print("Is dir:", os.path.isdir("config_files"))
print("Is file:", os.path.isfile("config_files"))

Exists: True
Is dir: True
Is file: False


In [22]:
open("config_files/test.txt", "w").close()

In [27]:
import os
from fabric import Connection

c = Connection('tfessler@rainbow', connect_kwargs={"password": "Dl2oP1AjOjO6"})

local_dir = "config_files"
remote_dir = "/home/tfessler/config_files"

# Ensure remote directory exists
c.run(f"mkdir -p {remote_dir}")

# Upload each file
for file in os.listdir(local_dir):
    local_path = os.path.join(local_dir, file)
    if os.path.isfile(local_path):
        c.put(local_path, remote=f"{remote_dir}/{file}")
        print(f"Uploaded {file}")

Uploaded spectrumVH2O_M=0.6.yaml
Uploaded spectrumVH2O_M=0.8.yaml
Uploaded spectrumVH2O_M=1.2.yaml
Uploaded spectrumVH2O_M=1.4.yaml
Uploaded spectrumVH2O_M=1.6.yaml
Uploaded spectrumVH2O_M=1.8.yaml
Uploaded spectrumVH2O_M=1.yaml
Uploaded spectrumVH2O_M=2.yaml


In [30]:
os.makedirs("config_files_M", exist_ok=True)

FileExistsError: [WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'config_files_M'